In [17]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json
import threading
import time

In [2]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [3]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [4]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [18]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'lab.aimet.tech:9092'

In [19]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [20]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [8]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [22]:
def monitor_thread():
    for message in resultconsumer:
        print(message.value)

In [23]:
monitor = threading.Thread(target=monitor_thread, daemon=True)
monitor.start()

{'timestamp': 1742895211, 'vid': '6532203021', 'txid': 'TX05682', 'checksum': '-', 'code': 400, 'message': 'Incorrect Signature'}
{'timestamp': 1742895212, 'vid': '6532203021', 'txid': 'TX02632', 'checksum': '-', 'code': 400, 'message': 'Incorrect Signature'}
{'timestamp': 1742895213, 'vid': '6532203021', 'txid': 'TX04116', 'checksum': '-', 'code': 400, 'message': 'Incorrect Signature'}
{'timestamp': 1742895214, 'vid': '6532203021', 'txid': 'TX03456', 'checksum': '-', 'code': 400, 'message': 'Incorrect Signature'}
{'timestamp': 1742895223, 'vid': '6532203021', 'txid': 'TX03276', 'checksum': '-', 'code': 400, 'message': 'Incorrect Signature'}
{'timestamp': 1742895230, 'vid': '6532203021', 'txid': 'TX08781', 'checksum': '-', 'code': 400, 'message': 'Incorrect Signature'}
{'timestamp': 1742895235, 'vid': '6532203021', 'txid': 'TX03537', 'checksum': '-', 'code': 400, 'message': 'Incorrect Signature'}
{'timestamp': 1742895242, 'vid': '6532203021', 'txid': 'TX06429', 'checksum': '-', 'code':

In [24]:
token = "idkman"
vid = "6532203021"

print('Running TX Consumer')
for message in txconsumer:
    txid = message.value['txid']
    payer = message.value['payer']
    payee = message.value['payee']
    amount = message.value['amount']

    signature = gen_signature(txid, payer, payee, amount, token)
    submit_data = {'vid': vid, 'txid': txid, 'signature': signature}
    serialized_submit_data = serialize(submitschema, submit_data)

    producer.send('submit', serialized_submit_data)
    print(message.value)
    print(f"submit: {submit_data}")

Running TX Consumer
{'txid': 'TX05682', 'payer': 'A35556', 'payee': 'A59861', 'amount': 1421}
submit: {'vid': '6532203021', 'txid': 'TX05682', 'signature': '21553878a9fa0d74410f2009d6081287'}
{'txid': 'TX02632', 'payer': 'A78888', 'payee': 'A62586', 'amount': 1466}
submit: {'vid': '6532203021', 'txid': 'TX02632', 'signature': 'f7f03081fb6a8cb75678737653d62dab'}
{'txid': 'TX04116', 'payer': 'A20690', 'payee': 'A63671', 'amount': 189}
submit: {'vid': '6532203021', 'txid': 'TX04116', 'signature': 'bcb4926a9126b4398e4f477e0a268730'}
{'txid': 'TX03456', 'payer': 'A86747', 'payee': 'A11219', 'amount': 632}
submit: {'vid': '6532203021', 'txid': 'TX03456', 'signature': 'd0f1838b7ee415be93fea8fe0fcf1826'}
{'txid': 'TX03276', 'payer': 'A38698', 'payee': 'A32103', 'amount': 1497}
submit: {'vid': '6532203021', 'txid': 'TX03276', 'signature': 'c0866457fbb72ce3edf59c245b871a0a'}
{'txid': 'TX08781', 'payer': 'A49728', 'payee': 'A10711', 'amount': 1582}
submit: {'vid': '6532203021', 'txid': 'TX08781',

KeyboardInterrupt: 